#  Durational Ratios



## A. Import Intervals and Other Code

* See the Corpus Methods notebook for details on the various options for local and remote files


In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import pandas as pd
import re
import requests
import os
import glob as glob

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


### Full CRIM Corpus

* Here we omit various monophonic pieces and a few others for which there are errors.

```
piece_list = []
raw_prefix = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/"
URL = "https://api.github.com/repos/CRIM-Project/CRIM-online/git/trees/990f5eb3ff1e9623711514d6609da4076257816c"
piece_json = requests.get(URL).json()
```

* A list of files to exclude:

```
exclude_list = ['CRIM_Model_0003.mei', 'CRIM_Model_0004.mei', 'CRIM_Model_0005.mei', 'CRIM_Model_0006.mei', 'CRIM_Model_0007.mei','CRIM_Model_0022.mei', 'CRIM_Model_0028.mei', 'CRIM_Model_0035.mei', 'CRIM_Mass_0029_4.mei', 'CRIM_Mass_0049_2.mei','CRIM_Mass_0049_5.mei']
```

*  The following ensures that we don't try to analyze the 'Mass head only' files, which have no musical content:

```
pattern = 'CRIM_Mass_([0-9]{4}).mei
```

* Now the request for all the files


```for p in piece_json["tree"]:
p_name = p["path"]
if re.search(pattern, p_name):
    pass
elif p_name in exclude_list:
    pass
else:
    piece_list.append(raw_prefix + p["path"])
```

In [2]:
# this will pull ALL pieces from CRIM on Github
# Note that we exclude various monophonic pieces (which have no contrapuntal cadences)
# and also a few pieces that seem to throw errors for reasons we don't understand.
corpus_list = []
raw_prefix = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/"
URL = "https://api.github.com/repos/CRIM-Project/CRIM-online/git/trees/990f5eb3ff1e9623711514d6609da4076257816c"
piece_json = requests.get(URL).json()

# list of files to exclude
exclude_list = ['CRIM_Model_0003.mei', 'CRIM_Model_0004.mei', 'CRIM_Model_0005.mei', 'CRIM_Model_0006.mei', 
             'CRIM_Model_0007.mei',
            'CRIM_Model_0022.mei', 'CRIM_Model_0028.mei', 'CRIM_Model_0035.mei', 'CRIM_Mass_0029_4.mei', 
             'CRIM_Mass_0049_2.mei',
            'CRIM_Mass_0049_5.mei']

# this ensures that we don't try to analyze the 'Mass head only' files, which have no musical content

pattern = 'CRIM_Mass_([0-9]{4}).mei'

# and now the request for all the files
for p in piece_json["tree"]:
    p_name = p["path"]
    if re.search(pattern, p_name):
        pass
    elif p_name in exclude_list:
        pass
    else:
        corpus_list.append(raw_prefix + p["path"])

In [3]:
# short list for testing:

corpus_list = corpus_list[0:5]

In [4]:
# now pass the corpus_list to corpbase (which creates the music21 files)
corpus  = CorpusBase(corpus_list)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_1.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_2.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_4.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_5.mei


In [5]:
# set length of desired ngrams
n = 3
func = ImportedPiece.durationalRatios
list_of_dur_rats = corpus.batch(func=func, metadata=True)
list_of_dur_rats_rounded = []
for df in list_of_dur_rats:
    list_of_dur_rats_rounded.append(df.round(2))
func2 = ImportedPiece.melodic
list_of_mel = corpus.batch(func=func2, kwargs={'kind' : 'd'})
func3 = ImportedPiece.ngrams
list_of_ngrs = corpus.batch(func=func3, kwargs={'df' : list_of_mel, 'other' : list_of_dur_rats_rounded, 'n' : n})
combined_df = pd.concat(list_of_ngrs, ignore_index=False)
combined_df

,1,2,3,4,Composer,Title
6.0,"1_0.33, -2_2.0, 2_1.0","1_0.33, 1_2.0, 1_1.0","1_0.33, -2_2.0, 2_1.0","1_0.33, -4_2.0, 4_1.0",Pierre Colin,Missa Confitemini: Kyrie
8.0,"-2_2.0, 2_1.0, 2_0.75","1_2.0, 1_1.0, 3_0.75","-2_2.0, 2_1.0, 2_0.75","-4_2.0, 4_1.0, -2_1.5",Pierre Colin,Missa Confitemini: Kyrie
12.0,"2_1.0, 2_0.75, -2_0.33","1_1.0, 3_0.75, -2_0.17","2_1.0, 2_0.75, -2_0.33","4_1.0, -2_1.5, -5_0.33",Pierre Colin,Missa Confitemini: Kyrie
16.0,"2_0.75, -2_0.33, 2_1.0","3_0.75, -2_0.17, -2_1.0","2_0.75, -2_0.33, -2_2.0","-2_1.5, -5_0.33, 4_2.0",Pierre Colin,Missa Confitemini: Kyrie
19.0,"-2_0.33, 2_1.0, -3_1.0","-2_0.17, -2_1.0, -2_4.0","-2_0.33, -2_2.0, -2_0.5",NaN,Pierre Colin,Missa Confitemini: Kyrie
...,...,...,...,...,...,...
252.0,"-2_1.0, -2_2.0, -2_0.5",NaN,"-2_1.0, -2_1.0, -2_2.0","2_2.0, 2_1.0, -4_2.0",Pierre Colin,Missa Confitemini: Agnus Dei
253.0,NaN,"-2_0.33, -2_1.0, -2_1.0",NaN,NaN,Pierre Colin,Missa Confitemini: Agnus Dei
254.0,"-2_2.0, -2_0.5, 2_8.0","-2_1.0, -2_1.0, 2_4.0","-2_1.0, -2_2.0, -2_4.0","2_1.0, -4_2.0, 4_4.0",Pierre Colin,Missa Confitemini: Agnus Dei
255.0,NaN,"-2_1.0, 2_4.0, 1_4.0",NaN,NaN,Pierre Colin,Missa Confitemini: Agnus Dei


In [6]:
# set length of desired ngrams
n = 3
func = ImportedPiece.durationalRatios
list_of_dur_rats = corpus.batch(func=func, metadata=True)
list_of_dur_rats_rounded = []
for df in list_of_dur_rats:
    list_of_dur_rats_rounded.append(df.round(2).astype(str))
func2 = ImportedPiece.melodic
list_of_mel = corpus.batch(func=func2, kwargs={'kind' : 'd'})
func3 = ImportedPiece.ngrams
list_of_ngrs = corpus.batch(func=func3, kwargs={'df' : list_of_mel, 'other' : list_of_dur_rats_rounded, 'n' : n})
combined_df = pd.concat(list_of_ngrs, ignore_index=False)
combined_df

,1,2,3,4,Composer,Title
6.0,"1_0.33, -2_2.0, 2_1.0","1_0.33, 1_2.0, 1_1.0","1_0.33, -2_2.0, 2_1.0","1_0.33, -4_2.0, 4_1.0",Pierre Colin,Missa Confitemini: Kyrie
8.0,"-2_2.0, 2_1.0, 2_0.75","1_2.0, 1_1.0, 3_0.75","-2_2.0, 2_1.0, 2_0.75","-4_2.0, 4_1.0, -2_1.5",Pierre Colin,Missa Confitemini: Kyrie
12.0,"2_1.0, 2_0.75, -2_0.33","1_1.0, 3_0.75, -2_0.17","2_1.0, 2_0.75, -2_0.33","4_1.0, -2_1.5, -5_0.33",Pierre Colin,Missa Confitemini: Kyrie
16.0,"2_0.75, -2_0.33, 2_1.0","3_0.75, -2_0.17, -2_1.0","2_0.75, -2_0.33, -2_2.0","-2_1.5, -5_0.33, 4_2.0",Pierre Colin,Missa Confitemini: Kyrie
19.0,"-2_0.33, 2_1.0, -3_1.0","-2_0.17, -2_1.0, -2_4.0","-2_0.33, -2_2.0, -2_0.5",NaN,Pierre Colin,Missa Confitemini: Kyrie
...,...,...,...,...,...,...
252.0,"-2_1.0, -2_2.0, -2_0.5",NaN,"-2_1.0, -2_1.0, -2_2.0","2_2.0, 2_1.0, -4_2.0",Pierre Colin,Missa Confitemini: Agnus Dei
253.0,NaN,"-2_0.33, -2_1.0, -2_1.0",NaN,NaN,Pierre Colin,Missa Confitemini: Agnus Dei
254.0,"-2_2.0, -2_0.5, 2_8.0","-2_1.0, -2_1.0, 2_4.0","-2_1.0, -2_2.0, -2_4.0","2_1.0, -4_2.0, 4_4.0",Pierre Colin,Missa Confitemini: Agnus Dei
255.0,NaN,"-2_1.0, 2_4.0, 1_4.0",NaN,NaN,Pierre Colin,Missa Confitemini: Agnus Dei


In [16]:
# now just the mel
n = 3
func = ImportedPiece.durationalRatios
list_of_dur_rats = corpus.batch(func=func, metadata=True)
list_of_dur_rats_rounded = []
for df in list_of_dur_rats:
    list_of_dur_rats_rounded.append(df.round(2))
func2 = ImportedPiece.melodic
list_of_mel = corpus.batch(func=func2, kwargs={'kind' : 'd'})
func3 = ImportedPiece.ngrams
list_of_mel_ngrs = corpus.batch(func=func3, kwargs={'df' : list_of_mel, 'n' : n})
combined_mel_dfs = pd.concat(list_of_mel_ngrs, ignore_index=False)
combined_mel_dfs = combined_mel_dfs.dropna(how="all", subset=combined_mel_dfs.columns[:-3])
combined_mel_dfs

,1,2,3,4,Composer,Title
6.0,"1, -2, 2","1, 1, 1","1, -2, 2","1, -4, 4",Pierre Colin,Missa Confitemini: Kyrie
8.0,"-2, 2, 2","1, 1, 3","-2, 2, 2","-4, 4, -2",Pierre Colin,Missa Confitemini: Kyrie
12.0,"2, 2, -2","1, 3, -2","2, 2, -2","4, -2, -5",Pierre Colin,Missa Confitemini: Kyrie
16.0,"2, -2, 2","3, -2, -2","2, -2, -2","-2, -5, 4",Pierre Colin,Missa Confitemini: Kyrie
19.0,"-2, 2, -3","-2, -2, -2","-2, -2, -2",NaN,Pierre Colin,Missa Confitemini: Kyrie
...,...,...,...,...,...,...
250.0,"4, -2, -2","3, -2, -2","4, -2, -2",NaN,Pierre Colin,Missa Confitemini: Agnus Dei
252.0,"-2, -2, -2",NaN,"-2, -2, -2","2, 2, -4",Pierre Colin,Missa Confitemini: Agnus Dei
253.0,NaN,"-2, -2, -2",NaN,NaN,Pierre Colin,Missa Confitemini: Agnus Dei
254.0,"-2, -2, 2","-2, -2, 2","-2, -2, -2","2, -4, 4",Pierre Colin,Missa Confitemini: Agnus Dei


In [18]:

# and just dur
n = 3
func = ImportedPiece.durationalRatios
list_of_dur_rats = corpus.batch(func=func, metadata=True)
list_of_dur_rats_rounded = []
for df in list_of_dur_rats:
    list_of_dur_rats_rounded.append(df.round(2).astype(str))
func2 = ImportedPiece.ngrams
list_of_dur_ngrs = corpus.batch(func=func2, kwargs={'df' : list_of_dur_rats_rounded, 'n' : n})
combined_dur_dfs = combined_dur_dfs.dropna(how="all", subset=combined_dur_dfs.columns[:-3])
combined_dur_dfs

,1,2,3,4,Composer,Title
6.0,"0.33, 2.0, 1.0","0.33, 2.0, 1.0","0.33, 2.0, 1.0","0.33, 2.0, 1.0",Pierre Colin,Missa Confitemini: Kyrie
8.0,"2.0, 1.0, 0.75","2.0, 1.0, 0.75","2.0, 1.0, 0.75","2.0, 1.0, 1.5",Pierre Colin,Missa Confitemini: Kyrie
12.0,"1.0, 0.75, 0.33","1.0, 0.75, 0.17","1.0, 0.75, 0.33","1.0, 1.5, nan",Pierre Colin,Missa Confitemini: Kyrie
16.0,"0.75, 0.33, nan","0.75, 0.17, 1.0","0.75, 0.33, nan","1.5, nan, nan",Pierre Colin,Missa Confitemini: Kyrie
19.0,"0.33, nan, 1.0","0.17, 1.0, 4.0","0.33, nan, 2.0","nan, nan, nan",Pierre Colin,Missa Confitemini: Kyrie
...,...,...,...,...,...,...
252.0,"1.0, nan, 2.0","nan, 0.33, 1.0","1.0, nan, 1.0","2.0, nan, 1.0",Pierre Colin,Missa Confitemini: Agnus Dei
253.0,"nan, 2.0, nan","0.33, 1.0, 1.0","nan, 1.0, nan","nan, 1.0, nan",Pierre Colin,Missa Confitemini: Agnus Dei
254.0,"2.0, nan, nan","1.0, 1.0, 4.0","1.0, nan, 2.0","1.0, nan, 2.0",Pierre Colin,Missa Confitemini: Agnus Dei
255.0,"nan, nan, 0.5","1.0, 4.0, nan","nan, 2.0, nan","nan, 2.0, nan",Pierre Colin,Missa Confitemini: Agnus Dei


In [26]:
df2 = pd.merge(combined_mel_dfs, combined_dur_dfs, left_index=True, right_index=True)
df2.fillna('')

,1_x,2_x,3_x,4_x,Composer_x,Title_x,1_y,2_y,3_y,4_y,Composer_y,Title_y
3.0,"2, 2, 2",,,,Pierre Colin,Missa Confitemini: Sanctus,"0.33, 1.0, 1.0","nan, nan, nan","nan, nan, nan","nan, nan, nan",Pierre Colin,Missa Confitemini: Sanctus
4.0,"2, 2, 2",,,,Pierre Colin,Missa Confitemini: Sanctus,"1.0, 1.0, 3.0","nan, nan, nan","nan, nan, nan","nan, nan, nan",Pierre Colin,Missa Confitemini: Sanctus
4.0,"2, 2, 2",,,,Pierre Colin,Missa Confitemini: Sanctus,"1.5, nan, 0.33","nan, 0.5, nan","nan, nan, nan","nan, 1.0, nan",Pierre Colin,Missa Confitemini: Agnus Dei
5.0,"2, 2, 2",,,,Pierre Colin,Missa Confitemini: Sanctus,"1.0, 3.0, nan","nan, nan, 0.5","nan, nan, nan","nan, nan, nan",Pierre Colin,Missa Confitemini: Sanctus
6.0,"1, -2, 2","1, 1, 1","1, -2, 2","1, -4, 4",Pierre Colin,Missa Confitemini: Kyrie,"0.33, 2.0, 1.0","0.33, 2.0, 1.0","0.33, 2.0, 1.0","0.33, 2.0, 1.0",Pierre Colin,Missa Confitemini: Kyrie
...,...,...,...,...,...,...,...,...,...,...,...,...
1047.0,"-3, 4, 1",,,,Pierre Colin,Missa Confitemini: Credo,"1.0, 2.0, 1.0","nan, 1.0, nan","nan, 1.0, nan","nan, 1.0, nan",Pierre Colin,Missa Confitemini: Credo
1048.0,"4, 1, -2","3, 1, -3","4, -2, -2","-3, 5, 2",Pierre Colin,Missa Confitemini: Credo,"2.0, 1.0, 1.0","1.0, nan, 0.5","1.0, nan, 0.5","1.0, nan, 0.5",Pierre Colin,Missa Confitemini: Credo
1050.0,"1, -2, -2",,,,Pierre Colin,Missa Confitemini: Credo,"1.0, 1.0, 2.0","nan, 0.5, 1.0","nan, 0.5, 1.0","nan, 0.5, 1.0",Pierre Colin,Missa Confitemini: Credo
1052.0,"-2, -2, -2","1, -3, 1","-2, -2, -2","5, 2, -4",Pierre Colin,Missa Confitemini: Credo,"1.0, 2.0, nan","0.5, 1.0, 2.0","0.5, 1.0, 2.0","0.5, 1.0, 2.0",Pierre Colin,Missa Confitemini: Credo


In [ ]:
# Select a prefix:
# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/' 


# Add your filename here
mei_file = 'CRIM_Model_0008.mei'

# These join the strings and import the piece
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

In [ ]:
dr = piece.durationalRatios().round(3)
mel = piece.melodic(kind='d')
ng = piece.ngrams(df=mel, other=dr,  n=1)
ng2 = piece.ngrams(df=mel, other=dr, n=2)
ng2